**Сначала на основе обучающей выборки строятся эмбеддинги, используя fasttext. Для каждого названия организации строится один вектор, равный сумме эмбеддингов слов, из которых состоит название. Каждому роду занятий ставится в соответствие номер, который и должна предсказывать модель. Т.е. получаем задачу, где вектор признаков объекта - его эмбеддинг, а метка класса - номер рода деятельности. Названия родов деятельности добавлены о обучающую выборку. По обучающей выборке обучаем случайный лес из решающих деревьев, полученная модель делает предсказание.**

In [1]:
file = open('names_and_rubrics_learn.tsv', 'r')
train = [line.strip().split('\t') for line in file]
file.close()

In [2]:
import fasttext

In [3]:
#model = fasttext.skipgram('names_and_rubrics_learn.tsv', 'model')
model = fasttext.load_model('model.bin')

In [5]:
file2 = open('names_and_rubrics_test_no_rubric.tsv', 'r')
test = [line.strip() for line in file2]
file2.close()

In [6]:
import numpy as np

In [7]:
from collections import defaultdict
rubric_id = defaultdict()
id_rubric = defaultdict()
id_ = 0
for i in range(len(train)):
    if train[i][1] not in rubric_id:
        rubric_id[train[i][1]] = id_
        id_rubric[id_] = train[i][1]
        id_ += 1

In [15]:
train_embed = defaultdict()
train_X = np.zeros((len(train) + len(rubric_id), len(model['hello'])))

for i in range(len(train)):
    if rubric_id[train[i][1]] not in train_embed:
        train_embed[rubric_id[train[i][1]]] = np.zeros(len(model['hello']))
    x_words = train[i][0].split(' ')
    for x_w in x_words:
        train_embed[rubric_id[train[i][1]]] += np.array(model[x_w])
        train_X[i, :] += np.array(model[x_w])
for i in range(len(id_rubric)):
    x_words = id_rubric[i].split(' ')
    for x_w in x_words:
        train_embed[i] += np.array(model[x_w])
        train_X[i + len(train), :] += np.array(model[x_w])

In [16]:
test_X = np.zeros((len(test), len(model['hello'])))
for i in range(len(test)):
    test_words = test[i].split(' ')
    for x_w in test_words:
        test_X[i, :] += np.array(model[x_w])

In [17]:
train_y = np.zeros(len(train) + len(id_rubric), dtype='int')
for i in range(len(train)):
    train_y[i] = rubric_id[train[i][1]]
for i in range(len(id_rubric)):
    train_y[i + len(train)] = i

In [18]:
def cosine_similarity(a, b):
    return np.sum(a * b) / np.sqrt(np.sum(a * a) * np.sum(b * b))

In [26]:
import datetime
corr = 0
cur = datetime.datetime.now()
for i in range(min(train_y.shape[0], 10000)):
    sims = [cosine_similarity(train_X[i], train_embed[j]) for j in range(len(rubric_id))]
    pred = np.argmax(sims)
    if pred == train_y[i]:
        corr += 1

In [27]:
print('Accuracy on train set', corr / train_y.shape[0])

Accuracy on train set 0.007196704387400456


In [29]:
import datetime
corr = 0
cur = datetime.datetime.now()
for i in range(len(rubric_id)):
    sims = [cosine_similarity(train_X[i + train_X.shape[0]], train_embed[j]) for j in range(len(rubric_id))]
    pred = np.argmax(sims)
    if pred == train_y[i + train_X.shape[0]]:
        corr += 1

IndexError: index 301249 is out of bounds for axis 0 with size 301249

In [ ]:
print('Accuracy on rubrics', corr / train_y.shape[0])

In [14]:
import xgboost
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from sklearn.neighbors import KNeighborsClassifier

#clf = KNeighborsClassifier(n_jobs = -1)
#clf = xgboost.XGBClassifier(n_jobs=-1)
#clf = RandomForestClassifier(n_estimators=100, max_depth = 5, verbose=2, n_jobs=-1)
clf = AdaBoostClassifier(n_estimators=5)

In [24]:
del model
del test
del train

In [ ]:
clf.fit(train_X, train_y)

In [ ]:
pred = clf.predict(test_X)
out = open('prediction.tsv', 'w')
for i in pred:
    out.write(id_rubric[i] + '\n')
out.close()

In [ ]:
del pred

In [ ]:
train_pred = clf.predict(train_X)
from sklearn.metrics import accuracy_score
print('Accuracy on train set', accuracy_score(train_pred, train_y))